# Install lib

In [1]:
!pip install -q llama-cpp-python huggingface-hub gradio sympy scipy pulp numpy matplotlib

Cell 1: Imports

In [2]:
# Essential libraries
import gradio as gr                     # beautiful chat UI
from huggingface_hub import hf_hub_download  # GGUF model download
from llama_cpp import Llama             # local LLM inference
import sympy as sp                      # symbolic mathematics
from sympy import symbols, sympify, solve, diff, integrate, latex, Matrix
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import re                               # expression cleaning
import json                             # not used yet, but future-proof
from scipy.optimize import minimize     # numerical optimization
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value

Cell 2: Model Download & Load
Original - seedha Qwen2.5-1.5B-Q5_K_M load kiya

In [3]:
# Load lightweight local math-capable LLM
MODEL_REPO = "Qwen/Qwen2.5-1.5B-Instruct-GGUF"
MODEL_FILENAME = "qwen2.5-1.5b-instruct-q5_k_m.gguf"

print("Downloading model from Hugging Face (first time may take a while)...")
model_path = hf_hub_download(repo_id=MODEL_REPO, filename=MODEL_FILENAME)

print("Initializing llama.cpp backend...")
math_llm = Llama(
    model_path=model_path,
    n_gpu_layers=0,           # CPU only (safe for most laptops/Colab free)
    n_ctx=8192,               # long enough for math reasoning
    verbose=False
)
print("Math LLM ready!")

Initializing llama.cpp backend...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Math LLM ready!


Cell 3: Expression Cleaner (preprocess_math_expression)
Original - regex heavy tha

In [4]:
def clean_math_input(raw_text: str) -> str:
    """
    Safely converts natural language math input into valid SymPy expression.
    Removes command words, handles implicit multiplication, fixes power notation.
    """
    text = raw_text.strip().lower()

    # Remove instruction words
    text = re.sub(r'\b(solve|find|calculate|what is|integrate|differentiate|derivative|plot|graph|show)\b\s*', '', text, flags=re.I)

    # Remove trailing equals or =0
    text = re.sub(r'\s*(=|\bequals?\b|=0\s*$).*', '', text, flags=re.I)

    # Power: ^ → **
    text = text.replace('^', '**')

    # Implicit multiplication: 2x → 2*x, xy → x*y, 3π → 3*π
    text = re.sub(r'(\d)([a-zA-Zπ])', r'\1*\2', text)
    text = re.sub(r'([a-zA-Zπ])(\d)', r'\1*\2', text)
    text = re.sub(r'([a-zA-Z])([a-zA-Z])', r'\1*\2', text)

    # Fraction safety: 1/2x → (1/2)*x
    text = re.sub(r'(\d+)/(\d+)([a-zA-Z])', r'(\1/\2)*\3', text)

    # Common symbols
    text = text.replace('sqrt', 'sqrt').replace('pi', 'pi').replace('e', 'E')

    return text

Cell 4: MathSolver Class (sabse important part)
Original - MathSolver class tha


In [5]:
class IntelligentMathAssistant:
    """Handles both symbolic math (SymPy) and natural language explanation (LLM)."""

    def __init__(self, llm_engine):
        self.engine = llm_engine
        self.conversation_log = []  # store problem + answer pairs

    def detect_problem_category(self, query: str) -> str:
        """Ask LLM to classify the math question type very quickly."""
        classification_prompt = f"""Classify this math question into ONE word only:
equation | derivative | integral | optimization | matrix | graph | puzzle | other

Question: {query}

Answer:"""
        response = self.engine(classification_prompt, max_tokens=20, temperature=0.05)
        category = response['choices'][0]['text'].strip().lower()
        return category

    def try_symbolic_computation(self, raw_query: str, category: str):
        """Attempt exact symbolic solution using SymPy."""
        try:
            expr_str = clean_math_input(raw_query)
            x, y, z = sp.symbols('x y z')
            env = {
                'sin': sp.sin, 'cos': sp.cos, 'tan': sp.tan,
                'exp': sp.exp, 'ln': sp.ln, 'log': sp.log,
                'sqrt': sp.sqrt, 'pi': sp.pi, 'E': sp.E
            }
            expression = sp.sympify(expr_str, locals=env)

            if category == 'equation':
                solutions = sp.solve(expression, x)
                return solutions, ", ".join(str(s) for s in solutions)
            elif category == 'derivative':
                result = sp.diff(expression, x)
                return result, str(result)
            elif category == 'integral':
                result = sp.integrate(expression, x)
                return result, f"{result} + C"
            elif category == 'optimization':
                f = sp.lambdify(x, expression)
                res = minimize(f, x0=0)
                return res.x[0], f"Minimum found at x = {res.x[0]:.4f}, value = {res.fun:.4f}"
            elif category == 'matrix':
                # Very simple matrix handling
                matrix_match = re.search(r'\[\[.*?\]\]', expr_str)
                if matrix_match:
                    mat_data = eval(matrix_match.group())
                    M = Matrix(mat_data)
                    if 'det' in raw_query.lower():
                        return M.det(), str(M.det())
                    if 'inv' in raw_query.lower():
                        return M.inv(), latex(M.inv())
            return None, "Could not solve symbolically"
        except Exception as exc:
            return None, f"Symbolic computation failed: {str(exc)}"

    def generate_human_like_steps(self, question: str, symbolic_result: str = None):
        """Generate short, textbook-style step-by-step solution using LLM."""
        base_prompt = """Follow this exact style — short clear steps, no chit-chat, end with boxed final answer.

Example 1: 2^x + 2*2^x = 24
→ 3*2^x = 24
→ 2^x = 8
→ 2^x = 2^3
→ x = 3
\\boxed{3}

Example 2: Sides 5 and 12, find hypotenuse
→ c = √(5² + 12²)
→ c = √(25 + 144) = √169
→ c = 13
\\boxed{13}

Example 3: Derivative of x^x
→ y = x^x = e^(x ln x)
→ dy/dx = e^(x ln x) * (ln x + 1)
→ dy/dx = x^x (ln x + 1)
\\boxed{x^{x} (\\ln x + 1)}

Now solve:
"""
        if symbolic_result:
            base_prompt += f"Symbolic result to use: {symbolic_result}\n\n"
        base_prompt += f"Question: {question}\n\nAnswer (3-7 lines max):"

        response = self.engine(base_prompt, max_tokens=400, temperature=0.15)
        return response['choices'][0]['text'].strip()

    def generate_graph_if_needed(self, query: str):
        """Plot expression if it looks like a graphing request."""
        if not any(word in query.lower() for word in ['plot', 'graph', 'draw', 'curve']):
            return None
        try:
            expr_text = clean_math_input(query)
            expr = sp.sympify(expr_text)
            f = sp.lambdify(sp.symbols('x'), expr, 'numpy')
            x = np.linspace(-10, 10, 500)
            y = f(x)
            plt.figure(figsize=(7, 4.5))
            plt.plot(x, y, color='teal', linewidth=2.5, label=str(expr))
            plt.grid(True, alpha=0.3)
            plt.axhline(0, color='black', lw=0.8)
            plt.axvline(0, color='black', lw=0.8)
            plt.title(f"Graph of {expr}")
            plt.xlabel("x")
            plt.ylabel("y")
            buf = BytesIO()
            plt.savefig(buf, format='png', bbox_inches='tight')
            buf.seek(0)
            plt.close()
            return Image.open(buf)
        except:
            return None

    def process_question(self, question: str):
        category = self.detect_problem_category(question)
        sym_result, sym_text = self.try_symbolic_computation(question, category)
        explanation = self.generate_human_like_steps(question, sym_text if sym_result else None)
        graph = self.generate_graph_if_needed(question)
        self.conversation_log.append({"question": question, "response": explanation})
        return explanation, graph

Cell 5: Gradio Chat Interface

In [8]:
# Create the assistant AFTER model is loaded
solver = IntelligentMathAssistant(math_llm)

def chat_respond(message, history):
    if not message.strip():
        return "", history, None

    answer, plot_img = solver.process_question(message)

    # Better LaTeX handling for Gradio Markdown
    formatted_answer = answer.replace('\\(', '$').replace('\\)', '$') \
                             .replace('\\[', '$$').replace('\\]', '$$') \
                             .replace('\\boxed{', '**\\boxed{')  # bold boxed

    history = history + [[message, formatted_answer]]
    return "", history, plot_img

with gr.Blocks(css=".gradio-container {background-color: #f0f4f8;}") as demo:
    gr.Markdown("# Upgraded Math Solver Chat\nConcise steps • LaTeX + Boxed answers • Plots when needed")

    chatbot = gr.Chatbot(height=550, show_copy_button=True)
    textbox = gr.Textbox(placeholder="Ask anything math-related...", label="Your question")
    image_out = gr.Image(label="Graph (if generated)", height=350)

    textbox.submit(
        chat_respond,
        inputs=[textbox, chatbot],
        outputs=[textbox, chatbot, image_out]
    )

demo.launch(share=True, debug=True)  # debug=True shows errors in UI